In [ ]:
import librosa
import matplotlib.pyplot as plt 
import librosa.display as libDisp
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pandas as pd
import numpy as np
import joblib
from sklearn import svm
from sklearn.neural_network import MLPClassifier

# MLP Model
MLP_model = MLPClassifier(hidden_layer_sizes=100, solver='sgd', learning_rate_init=0.01, max_iter=500)

In [ ]:
notes = []
for i in range(5, 7):
    for note in ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']:
        notes.append(f'{note}{i}')

mapping = {i: notes[i] for i in range(len(notes))}

In [ ]:
def run_note(X, note_no, y):
  X_transformed = None
  targets = None
  note_name = mapping[note_no]
  targets = y[:, note_no]
  
  if len(np.unique(targets)) < 2:
    return X_transformed, targets

  lda = LinearDiscriminantAnalysis(n_components=1)
  X_transformed  = lda.fit_transform(X, targets)

  return X_transformed, targets

In [ ]:
# Train Model
train_acc = []
train_idx = ['1', '2', '4', '6', '7', '9']
for idx in train_idx:
  file_name = 'song' + idx
  a = np.load(file_name + '.npz', allow_pickle=True)

  b = a['arr_0']
  b = b.sum(axis=0)

  X = b['X']
  y = b['y']

  for i in range(len(notes)):
    X_t, y_cur = run_note(X, i, y)
    if np.any(X_t) != None:
      MLP_model.fit(X_t, y_cur) #Trains the model
      acc = MLP_model.score(X_t, y_cur) 
      train_acc.append(acc)

print('Done Training...')
print('Accuracy on Test Set', np.mean(train_acc))

Done Training...


In [ ]:
'''
  Save model
'''
filename = 'LDA_MLP_classification.sav'
joblib.dump(MLP_model, filename)

['LDA_MLP_classification.sav']

In [ ]:
'''
  Valid Data
'''
valid_acc = []
loaded_model = joblib.load('LDA_MLP_classification.sav')

def run_note(X, note_no, y):
  X_transformed = None
  targets = None
  note_name = mapping[note_no]
  targets = y[:, note_no]
  
  if len(np.unique(targets)) < 2:
    return X_transformed, targets

  lda = LinearDiscriminantAnalysis(n_components=1)
  X_transformed  = lda.fit_transform(X, targets)

  return X_transformed, targets


valid_idx = ['8', '10']
for idx in valid_idx:

  file_name = 'song' + idx
  a = np.load(file_name + '.npz', allow_pickle=True)

  b = a['arr_0']
  b = b.sum(axis=0)

  X = b['X']
  y = b['y']



  for i in range(len(notes)):
    X_t, y_cur = run_note(X, i, y)
    if np.any(X_t) != None:
      X_test = X_t[:,0].reshape((X_t[:,0].shape[0], 1))
      pred = loaded_model.predict(X_test)
      acc = loaded_model.score(X_test, y_cur) 
      valid_acc.append(acc)


print('Accuracy on Validation Set', np.mean(valid_acc))

0.957315862616637


In [ ]:
'''
  Test Data
'''

test_acc = []
loaded_model = joblib.load('LDA_MLP_classification.sav')

def run_note(X, note_no, y):
  X_transformed = None
  targets = None
  note_name = mapping[note_no]
  targets = y[:, note_no]
  
  if len(np.unique(targets)) < 2:
    # print(f"note {note_name} has no samples")
    return X_transformed, targets

  lda = LinearDiscriminantAnalysis(n_components=1)
  X_transformed  = lda.fit_transform(X, targets)

  return X_transformed, targets


test_idx = ['3', '5']
for idx in test_idx:

  file_name = 'song' + idx
  a = np.load(file_name + '.npz', allow_pickle=True)

  b = a['arr_0']
  b = b.sum(axis=0)

  X = b['X']
  y = b['y']


  # Test
  for i in range(len(notes)):
    X_t, y_cur = run_note(X, i, y)
    if np.any(X_t) != None:
      X_test = X_t[:,0].reshape((X_t[:,0].shape[0], 1))
      acc = loaded_model.score(X_test, y_cur)
      test_acc.append(acc)
      
print('Accuracy on Test Set', np.mean(test_acc))

0.9165952607637788


In [ ]:
'''
  Runs Demo to print out predicted notes from a song using MLP model
'''

idx = '5'
file_name = 'song' + idx
a = np.load(file_name + '.npz', allow_pickle=True)

b = a['arr_0']
b = b.sum(axis=0)

X = b['X']
y = b['y']


target_notes = []

def map_notes(X, note_no, y):
  note_name = mapping[note_no]
  targets = y[:, note_no]
  target_notes.append(targets)



for i in range(len(notes)):
  map_notes(X, i, y)


target_notes = np.array(target_notes)
# print(target_notes.shape)


gt_notes_played = []
for curr_note in range(target_notes.shape[1]):
  curr_note_played = []
  curr_pos = target_notes[:, curr_note]
  for j in range(len(curr_pos)):
    if curr_pos[j] == 1:
      curr_note_played.append(mapping[j])

  gt_notes_played.append(curr_note_played)





'''
  Test on Predict Data
'''

test_acc = []
loaded_model = joblib.load('LDA_MLP_classification.sav')

def run_note(X, note_no, y):
  X_transformed = None
  targets = None
  note_name = mapping[note_no]
  targets = y[:, note_no]
  
  # if len(np.unique(targets)) < 2:
  if len(np.unique(targets)) < 2:
    # print(f"note {note_name} has no samples")
    return X_transformed, targets

  lda = LinearDiscriminantAnalysis(n_components=1)
  X_transformed  = lda.fit_transform(X, targets)

  return X_transformed, targets


# test_idx = ['3', '5']
test_idx = ['5']

pred_notes = []
for idx in test_idx:

  file_name = 'song' + idx
  a = np.load(file_name + '.npz', allow_pickle=True)

  b = a['arr_0']
  b = b.sum(axis=0)

  X = b['X']
  y = b['y']


  # Test
  for i in range(len(notes)):
    X_t, y_cur = run_note(X, i, y)



    if np.any(X_t) != None:
      # X_test = X_t[:,0].reshape((X_t[:,0].shape[0], 1))
      X_test = X_t
      pred = loaded_model.predict(X_test)
      acc = loaded_model.score(X_test, y_cur) 

      pred_notes.append(pred)


    else:
      pred_notes.append(np.zeros((X_test.shape[0],)))


pred_notes = np.array(pred_notes)

pred_notes_played = []
for curr_note in range(pred_notes.shape[1]):
  curr_note_played = []
  curr_pos = pred_notes[:, curr_note]
  
  for j in range(len(curr_pos)):
    if curr_pos[j] == 1:
      curr_note_played.append(mapping[j])
      # print(mapping[j])
      # input('s')


  pred_notes_played.append(curr_note_played)

print((gt_notes_played))
print((pred_notes_played))   

a = gt_notes_played
b = (pred_notes_played)

res = np.where(np.equal(a, b))

# print(res)

[['D4', 'G5'], ['D4', 'A5'], ['A5'], ['A4', 'A5'], ['A4'], ['D5'], ['A4', 'D5'], ['D5', 'E5'], ['A4', 'D5'], ['D5'], ['D5', 'G5'], ['D5', 'A5'], ['D5', 'A5'], ['D5'], ['D5'], ['D4', 'G5'], ['D4', 'A5'], ['A4', 'A5'], ['D4', 'A4'], ['A4', 'B4'], ['B4'], ['D4', 'G5'], ['D4', 'A5'], ['D4', 'A4', 'A5'], ['A4', 'A5'], ['A4'], ['A4', 'D5'], ['A4', 'D5'], ['A4', 'D5', 'E5'], ['A4', 'D5', 'E5'], ['A4', 'D5'], ['A4', 'D5', 'G5'], ['D5', 'G5'], ['D5', 'A5'], ['A4', 'D5', 'A5'], ['A4', 'A5'], ['A4'], ['A4'], ['D4'], ['D4'], ['D4', 'A4'], ['D4', 'A4'], ['D4', 'B4'], ['B4'], ['B4'], ['E4'], ['A4'], ['A4'], ['A4'], ['A4'], ['A4'], ['E4'], ['B4'], ['B4'], ['B4'], ['D4'], ['A4'], ['A4'], ['A4'], ['A4'], ['A4'], ['D4'], ['B4'], ['B4'], ['F#4'], ['B4'], ['B4'], ['B4'], ['B4'], ['B4'], ['E4'], ['A4'], ['A4'], ['B5'], ['D4', 'B5'], ['A4', 'B5'], ['A4'], ['A4', 'B5'], ['A4', 'A5'], ['A4'], ['D4'], ['A4'], ['G4'], ['A4'], ['A4'], ['A4'], ['F#5'], ['F#5'], ['D4', 'D5'], ['D5'], ['E4', 'D5'], ['E4', 'G4'], ['

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:128: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
